# QAQC order and single function testing

## Import libraries

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import glob

%matplotlib inline
%load_ext autoreload
%autoreload 2

## Import QAQC libraries

In [ ]:
import os
import tempfile
import argparse 

# Import all qaqc script functions
try:
    from qaqc_plot import *
    from qaqc_utils import *
    from qaqc_wholestation import *
    from qaqc_logic_checks import *
    from qaqc_buoy_check import *
    from qaqc_frequent import *
    from qaqc_unusual_gaps import *
    from qaqc_unusual_large_jumps import *
    from qaqc_climatological_outlier import *
    from qaqc_unusual_streaks import *
except Exception as e:
    print("Error importing qaqc script: {}".format(e))

# Import qaqc stage calc functions
try:
    from QAQC_pipeline import *
except:
    print("Error importing QAQC_pipeline.py")
    
if not os.path.exists("./qaqc_figs"):
    os.makedirs("./qaqc_figs")

## Testing on a single network / station

### Load file and convert to pandas df 

In [ ]:
""" Aqaqc_world_recordtions from QAQC libraries can be used here.
    For the qaqc pipeline, we need the pandas df in the same format used in it
"""

network = "VCAPCD"
rawdir, cleandir, qaqcdir, mergedir = get_file_paths(network)
print(rawdir)
print(cleandir)
print(qaqcdir)
print(mergedir)

In [ ]:
""" We can test in the same way that the pipeline, 
    take a network and subsample an station
"""

files, stations = read_network_files(network, cleandir)
stations_sample = stations.sample(1)
station = stations_sample.iloc[0]
print(station)

In [ ]:
""" We could load station files from the 
    s3 bucket
"""
file_name = cleandir + station + ".nc"
fs = s3fs.S3FileSystem()
aws_url = "s3://wecc-historical-wx/"+file_name

with fs.open(aws_url) as fileObj:
    t0 = time.time()
    ds = xr.open_dataset(fileObj).load()
    ds = ds.drop_duplicates(dim="time")

In [ ]:
ds

In [ ]:
""" Or we can test on a locally stored station 
    file directly
"""
ds = xr.open_dataset('Train_Files/LOXWFO_OX1MB.nc')
ds

In [ ]:
""" Then, the idea is to convert to pandas df in the 
    format needed for the qaqc pipeline
"""
df, MultiIndex, attrs, var_attrs = qaqc_ds_to_df(ds)
df.head(5)

### Test QAQC pipeline of single functions

After converting to df in the same format than the pipeline, we could test the whole pipeline, or single functions

#### Testing single functions

In [ ]:
new_df = df.copy()

In [ ]:
new_df = qaqc_world_record(new_df)

In [ ]:
new_df = qaqc_elev_range(new_df)

In [ ]:
new_df = qaqc_unusual_repeated_streaks(new_df, plot=True, local=True, verbose=True)

In [ ]:
new_df = qaqc_climatological_outlier(new_df, plot=True, verbose=True)

#### Testing the whole pipeline

In [ ]:
# Set up error handling.
errors, end_api, timestamp = setup_error_handling()

In [ ]:
df, attrs, var_attrs = run_qaqc_pipeline(ds, network, file_name, errors, 
                                         station, end_api, rad_scheme="remove_zeros",
                                         verbose=True, local=True)

In [ ]:
### Or we can always run the same way that the ALL_NETWORKS_qaqc.py does:
# whole_station_qaqc(network, cleandir, qaqcdir, rad_scheme="remove_zeros", 
#                    verbose=True, local=True)

In [ ]:
""" Aqaqc_world_recordtions from QAQC libraries can be used here.
    For the qaqc pipeline, we need the pandas df in the same format used in it
"""

network = "ASOSAWOS"
rawdir, cleandir, qaqcdir, mergedir = get_file_paths(network)
print(rawdir)
print(cleandir)
print(qaqcdir)
print(mergedir)

In [ ]:
""" We can test in the same way that the pipeline, 
    take a network and subsample an station
"""

files, stations = read_network_files(network, cleandir)
stations_sample = stations.sample(1)
station = stations_sample.iloc[0]
print(station)

In [ ]:
pwd

- ASOSAWOS_74718503144
- ASOSAWOS_74917900392 

In [6]:
""" Or we can test on a locally stored station 
    file directly
"""
ds = xr.open_dataset('Train_Files/ASOSAWOS_74917900392.nc')
ds

<xarray.Dataset>
Dimensions:          (station: 1, time: 373657)
Coordinates:
  * time             (time) datetime64[ns] 2009-02-01 ... 2022-08-31T23:55:00
    lat              (station, time) float64 ...
    lon              (station, time) float64 ...
  * station          (station) object 'ASOSAWOS_74917900392'
Data variables: (12/20)
    ps               (station, time) float64 ...
    tas              (station, time) float64 ...
    tdps             (station, time) float64 ...
    pr               (station, time) float64 ...
    sfcWind          (station, time) float64 ...
    sfcWind_dir      (station, time) float64 ...
    ...               ...
    pr_qc            (station, time) object ...
    pr_duration      (station, time) timedelta64[ns] ...
    pr_depth_qc      (station, time) float64 ...
    sfcWind_qc       (station, time) object ...
    sfcWind_method   (station, time) object ...
    sfcWind_dir_qc   (station, time) object ...
Attributes: (12/18)
    title:                 ASOS/AWOS cleaned
    institution:           Eagle Rock Analytics / Cal Adapt
    source:                
    history:               ASOSAWOS_clean.py script run on 02-06-2023, 23:43:...
    comment:               Intermediate data product: may not have been subje...
    license:               
    ...                    ...
    CALL:                  CVH
    Networks:              AWOS
    thermometer_height_m:  nan
    anemometer_height_m:   nan
    barometer_elev_m:      nan
    raw_files_merged:      14

In [ ]:
# ds.ps_altimeter.plot()

In [7]:
""" Then, the idea is to convert to pandas df in the 
    format needed for the qaqc pipeline
"""
df, MultiIndex, attrs, var_attrs = qaqc_ds_to_df(ds)
df.head(2)

/Users/hector/anaconda3/envs/era_py39/lib/python3.9/site-packages/pandas/core/arrays/timedeltas.py:908: RuntimeWarning: invalid value encountered in cast
  base = data.astype(np.int64)
/Users/hector/anaconda3/envs/era_py39/lib/python3.9/site-packages/pandas/core/arrays/timedeltas.py:912: RuntimeWarning: invalid value encountered in cast
  data = (base * m + (frac * m).astype(np.int64)).view("timedelta64[ns]")


,time,ps,tas,tdps,pr,sfcWind,sfcWind_dir,elevation,qaqc_process,ps_qc,...,tdps_eraqc,pr_eraqc,sfcWind_eraqc,sfcWind_dir_eraqc,elevation_eraqc,ps_altimeter_eraqc,pr_duration_eraqc,anemometer_height_m,thermometer_height_m,station
0,2009-02-01 00:00:00,NaN,294.15,280.15,NaN,4.6,320.0,72.0,V020,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,ASOSAWOS_74917900392
1,2009-02-01 00:15:00,NaN,292.15,280.15,NaN,5.7,320.0,72.0,V020,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,ASOSAWOS_74917900392


In [ ]:
# new_df = qaqc_unusual_large_jumps(df, plot=True, local=True, verbose=True)

In [ ]:
# new_df = qaqc_unusual_repeated_streaks(df, plot=True, local=True, verbose=True)

In [ ]:
# new_df = qaqc_unusual_large_jumps(new_df, plot=True, local=True, verbose=True)

In [10]:
new_df = qaqc_unusual_gaps(df, plots=True, verbose=True)

KeyboardInterrupt: 